In [ ]:
import sys
{sys.executable}

{'/usr/bin/python3'}

In [ ]:
import torch
torch.__version__

'1.9.0+cu102'

In [ ]:
!pip install jdc

In [ ]:
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import jdc
import json
from collections import defaultdict, OrderedDict
import math
import numpy as np

torch.manual_seed(1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
def argmax(vec):
    _, idx = torch.max(vec, 1)
    return idx.item()

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [ ]:
with open('allData.json', 'r') as f:
    lines = f.readlines()

print(json.loads(lines[1]))

{'id': 2503, 'data': '@slycbiskut way much rain here gettin colder nothing like guys get', 'label': [[0, 11, 'B-PERSON'], [26, 30, 'B-LOC'], [38, 44, 'B-EVENT']]}


In [ ]:
# ix_to_label = {8: 'O', 7: 'I-LOC', 6: 'B-LOC' , 5: 'I-PER', 4: 'B-PER', 3: 'I-EVENT', 2: 'B-EVENT'}

data = []
vocab = set()

for line in lines:
  # print(line)
  words = []
  labels = []
  injson = json.loads(line)
  # print(injson)
  annots = injson['label']
  text = injson['data']
  split_text = text.split(" ")
  orddict = OrderedDict({})
  for ann in annots:
    orddict[text[ann[0]:(ann[1]+1)].rstrip()] = ann[2]
  # print(orddict)

  for word in split_text:
    words.append(word)
    vocab.add(word)
    labels.append(orddict.get(word, 'O'))
  data.append((words, labels))
  
print(data[1])       

(['@slycbiskut', 'way', 'much', 'rain', 'here', 'gettin', 'colder', 'nothing', 'like', 'guys', 'get'], ['B-PERSON', 'O', 'O', 'O', 'B-LOC', 'O', 'B-EVENT', 'O', 'O', 'O', 'O'])


In [ ]:
import csv

In [ ]:
training_data = []
lines = []
with open('tagged_lines.txt', 'r') as f:
  all_lines = csv.reader(f, delimiter = "\t")
  lines = list(all_lines)

In [ ]:
len(lines)

31831

In [ ]:
lines[29107] = ['Tite', 'O']

In [ ]:
print((lines[29107]))

['Tite', 'O']


In [ ]:
sentence = []
tags = []
for i in range(len(lines)):
  if lines[i] == []:
    training_data.append((sentence, tags))
    sentence = []
    tags = []
    continue
  sentence.append(lines[i][0])
  tags.append(lines[i][1])

In [ ]:
print(len(training_data))

2109


In [ ]:
print(len(data))

5390


In [ ]:
training_data = training_data + data

In [ ]:
print(training_data[0][0])

['O', 'est', 'le', 'rapport', 'avec', 'les', 'philosophes', 'ici', 'On', 'traite', 'un', 'sujet', 'serieu', 'cyber', 'harclement', 'mais', 'malheureusement', 'de', 'nos', 'jours', 'il', 'faut', 'attendre', 'que', 'quelquun', 'se', 'suicide', 'pour', 'rendre', 'compte', 'du', 'rel', 'problme', 'cest']


In [ ]:
vocab = set()

for i in range(len(training_data)):
  for word in training_data[i][0]:
    vocab.add(word)

In [ ]:
num_train = math.floor(len(data) * 0.7) 
training_data, test_data = data[:num_train], data[num_train:]


In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 16 # 8
HIDDEN_DIM = 16  # 16
MINIBATCH_SIZE = 2
LEARNING_WEIGHT = 0.05
WEIGHT_DECAY = 1e-4

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        self.hidden = self.init_hidden()

In [ ]:
%%add_to BiLSTM_CRF

def init_hidden(self):
    return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
            torch.randn(2, 1, self.hidden_dim // 2).to(device))

In [ ]:
%%add_to BiLSTM_CRF

def _forward_alg(self, feats):
    
    init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
    init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
    forward_var = init_alphas
    for feat in feats:
        alphas_t = []  
        for next_tag in range(self.tagset_size):
            emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
            trans_score = self.transitions[next_tag].view(1, -1)
            next_tag_var = forward_var + trans_score + emit_score
            alphas_t.append(log_sum_exp(next_tag_var).view(1))
        forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

In [ ]:
%%add_to BiLSTM_CRF

def _get_lstm_features(self, sentence):
    self.hidden = self.init_hidden()
    embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, self.hidden = self.lstm(embeds, self.hidden)
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
    lstm_feats = self.hidden2tag(lstm_out)
    return lstm_feats

In [ ]:
%%add_to BiLSTM_CRF

def _score_sentence(self, feats, tags):
    score = torch.zeros(1).to(device)
    tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
    for i, feat in enumerate(feats):
        score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
    return score

In [ ]:
%%add_to BiLSTM_CRF

def _viterbi_decode(self, feats):
    backpointers = []
    init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0
    forward_var = init_vvars
    for feat in feats:
        bptrs_t = []  
        viterbivars_t = []  

        for next_tag in range(self.tagset_size):
            next_tag_var = forward_var + self.transitions[next_tag]
            best_tag_id = argmax(next_tag_var)
            bptrs_t.append(best_tag_id)
            viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
        forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1).to(device)
        backpointers.append(bptrs_t)

    terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
        best_tag_id = bptrs_t[best_tag_id]
        best_path.append(best_tag_id)
    start = best_path.pop()
    assert start == self.tag_to_ix[START_TAG] 
    best_path.reverse()
    return path_score, best_path

In [ ]:
%%add_to BiLSTM_CRF

def neg_log_likelihood(self, sentence, tags):
    feats = self._get_lstm_features(sentence)
    forward_score = self._forward_alg(feats)
    gold_score = self._score_sentence(feats, tags)
    return forward_score - gold_score

In [ ]:
%%add_to BiLSTM_CRF

def forward(self, sentence):
    lstm_feats = self._get_lstm_features(sentence)
    score, tag_seq = self._viterbi_decode(lstm_feats)
    return score, tag_seq

In [ ]:
def remove_punct(text):
    punct = list(".,()-")
    for p in punct:
        text = text.replace(p, '')
    return text
    
text = remove_punct(text)

In [ ]:
word_to_ix = {k: v for (k, v) in zip(vocab, range(len(vocab)))}
tag_to_ix = {'B-EVENT': 0, 'I-EVENT': 1, 'B-PERSON':2, 'I-PERSON':3, 'B-LOC':4, 'I-LOC':5, 'O':6, START_TAG: 7, STOP_TAG:8}
ix_to_tag = {6: 'O', 5: 'I-LOC', 4: 'B-LOC' , 3: 'I-PERSON', 2: 'B-PERSON', 1: 'I-EVENT', 0: 'B-EVENT'}


In [ ]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
model = model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_WEIGHT, weight_decay=WEIGHT_DECAY)

In [ ]:
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', training_data[0][1])
    print(training_data[0][0])

Prediction:    ['I-PERSON', 'O', 'I-LOC', 'I-EVENT', 'B-PERSON', 'I-LOC', 'I-EVENT', 'B-PERSON', 'I-EVENT', 'B-PERSON', 'I-LOC', 'I-EVENT', 'B-PERSON', 'I-LOC', 'I-EVENT', 'B-PERSON', 'I-LOC', 'I-EVENT', 'B-PERSON']
Ground truth:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT']
['wwwwe', 'mmmmuch', 'timmmme', 'little', 'wwwwith', 'thats', 'wwwwhat', 'commmmes', 'wwwwith', 'mmmmidcaste', 'anything', 'commmmplain', 'lot', 'people', 'things', 'wwwworse', 'Oh', 'boy', 'depressed']


In [ ]:
%%time

losses = []
for epoch in range(10):  
    for sentence, tags in training_data:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        sentence_in, targets = sentence_in.to(device), targets.to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print("Epoch: {} Loss: {}".format(epoch+1, np.mean(losses)))

In [ ]:
torch.save(model.state_dict(), 'model_70.pth')

In [ ]:
model.eval() 

test_datum = training_data[0][0]
test_text = training_data[0][1]

with torch.no_grad():
    precheck_sent = prepare_sequence(test_datum, word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', test_text)
    print('Text: ', test_datum)

Prediction:    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT']
Ground truth:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-EVENT']
Text:  ['wwwwe', 'mmmmuch', 'timmmme', 'little', 'wwwwith', 'thats', 'wwwwhat', 'commmmes', 'wwwwith', 'mmmmidcaste', 'anything', 'commmmplain', 'lot', 'people', 'things', 'wwwworse', 'Oh', 'boy', 'depressed']


In [ ]:
model.eval() 
test_datum = test_data[198][0]
test_text = test_data[198][1]

with torch.no_grad():
    precheck_sent = prepare_sequence(test_datum, word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    print('Prediction:   ', [ix_to_tag[idx] for idx in pred])
    print('Ground truth: ', test_text)
    print('Text: ', test_datum)

Prediction:    ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Ground truth:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Text:  ['13', 'Reasons', 'trust', 'claim', '13', 'Reasons', 'Why', 'increased', 'youth', 'suicide', 'rates', 'Statistical', 'Modeling', 'Causal']


In [ ]:
model.eval() 

y_true = []
y_pred = []
with torch.no_grad():
  for i in range(len(test_data)):
    y_true.append(test_data[i][1])
    test_datum = test_data[i][0]
    precheck_sent = prepare_sequence(test_datum, word_to_ix)
    precheck_sent = precheck_sent.to(device)
    pred =  model(precheck_sent)[1]
    pred_tags = [ix_to_tag[idx] for idx in pred]
    y_pred.append(pred_tags)

In [ ]:
pip install seqeval

In [ ]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

In [ ]:
accuracy_score(y_true, y_pred)

0.9834037063101102

In [ ]:
f1_score(y_true, y_pred)

0.6854724964739068